****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

<h3>Top 10 countries with most projects</h3>

In [29]:
#import all the necessary packages
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
#load json file into a data frame and inspect it 
json_df = pd.read_json('data/world_bank_projects.json')
json_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 50 columns):
_id                         500 non-null object
approvalfy                  500 non-null int64
board_approval_month        500 non-null object
boardapprovaldate           500 non-null object
borrower                    485 non-null object
closingdate                 370 non-null object
country_namecode            500 non-null object
countrycode                 500 non-null object
countryname                 500 non-null object
countryshortname            500 non-null object
docty                       446 non-null object
envassesmentcategorycode    430 non-null object
grantamt                    500 non-null int64
ibrdcommamt                 500 non-null int64
id                          500 non-null object
idacommamt                  500 non-null int64
impagency                   472 non-null object
lendinginstr                495 non-null object
lendinginstrtype            495 non

Our target is top-10 most frequent values in the countryname column. Therefore, we can use pandas value_counts() which outputs the most frequent values in a column in descending order

In [44]:
#use value_counts() to determine countries with most projects
most_projects_countries = json_df.countryname.value_counts().head(10)
#display top-10 values 
most_projects_countries

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

<h3>Top 10 project themes</h3>

Let's inspect the'mjtheme_namecode'column

In [4]:
#output the 'mjtheme_namecode' column
json_df.mjtheme_namecode

0      [{u'code': u'8', u'name': u'Human development'...
1      [{u'code': u'1', u'name': u'Economic managemen...
2      [{u'code': u'5', u'name': u'Trade and integrat...
3      [{u'code': u'7', u'name': u'Social dev/gender/...
4      [{u'code': u'5', u'name': u'Trade and integrat...
5      [{u'code': u'6', u'name': u'Social protection ...
6      [{u'code': u'2', u'name': u'Public sector gove...
7      [{u'code': u'11', u'name': u'Environment and n...
8      [{u'code': u'10', u'name': u'Rural development...
9      [{u'code': u'2', u'name': u'Public sector gove...
10     [{u'code': u'10', u'name': u'Rural development...
11     [{u'code': u'10', u'name': u'Rural development...
12                       [{u'code': u'4', u'name': u''}]
13     [{u'code': u'5', u'name': u'Trade and integrat...
14     [{u'code': u'6', u'name': u'Social protection ...
15     [{u'code': u'10', u'name': u'Rural development...
16     [{u'code': u'10', u'name': u'Rural development...
17     [{u'code': u'8', u'name'

It contains nested data. Therefore, we have to use json_normalize to create a table from nested elements

In [11]:
#first, load data as a string 
json_string = json.load(open('data/world_bank_projects.json'))
#create a table with theme code using json_normalize
themes = json_normalize(json_string, 'mjtheme_namecode')
top_10_themes = themes['code'].value_counts().head(10)
top_10_themes

11    250
10    216
8     210
2     199
6     168
4     146
7     130
5      77
9      50
1      38
Name: code, dtype: int64

In [13]:
themes.head(20)

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


It seems like some names are missing. We can fill in the missing values using the code for each theme. First, we have to sort the existing theme table with respect to code, so each code will be equivalent to the name on the right. 

In [19]:
#sort the themes with respect to code name
themes = themes.sort_values(['code','name'])
themes.head(60)

,code,name
212,1,
363,1,
1024,1,
1114,1,
1437,1,
497,1,Economic management
2,1,Economic management
88,1,Economic management
175,1,Economic management
458,1,Economic management


It would be useful to propagate the missing values backward as the first rows of each code section are empty. However, these cells are empty (not NaN), which prevents us from using .fillna() function. Therefore, we need to replace all empty cells in the name column with NaN values (np.nan).

In [40]:
#replace empty cells with NaN
themes = themes.apply(lambda x: x.str.strip()).replace('', np.nan)
#check for replacement
themes.head(60)

,code,name
212,1,NaN
363,1,NaN
1024,1,NaN
1114,1,NaN
1437,1,NaN
497,1,Economic management
2,1,Economic management
88,1,Economic management
175,1,Economic management
458,1,Economic management


All empty cells were successfully replaced. Now, we can propagate the project names using .fillna()

In [41]:
#use .fillna() to replace NaN values 
themes = themes.fillna(method='bfill')
themes.head()

,code,name
212,1,Economic management
363,1,Economic management
1024,1,Economic management
1114,1,Economic management
1437,1,Economic management


Now, we can output the values of the top-10 project themes

In [43]:
top_10_project_themes = themes.name.value_counts().head(10)
top_10_project_themes

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64